## What is Numba?

a JIT (Just-in-Time) compiler for Python that:



- generates optimized machine code using LLVM (Low Level Virtual Machine) compiler infrastructure


- provides toolbox for different targets and execution models:
    - Single-threaded CPU, multi-threaded CPU, GPU
    - regular functions, "universal functions (ufuncs)" (array functions), etc


- integrates well with the Scientific Python stack


- with a few annotations, array-oriented and math-heavy Python code provides: 
 - speedup: 2x (compared to basic NumPy code) to 200x (compared to pure Python)
  - performance similar to C, C++, Fortran, without having to switch languages or Python interpreters


- is **totally awesome!**

## Basic Example

### Lazy Compilation

- Use `@jit` decorator
- Let Numba decide when and how to optimize

In [1]:
import numpy as np 
from numba import jit

In [2]:
@jit
def do_math(x, y):
    return x + y

In this mode:

- The compilation will be deferred until the first execution
- Numba will:
    - infer the argument types at call time
    - generate optimized code based on this information
- Numba will also be able to compile separate specializations depending on the input types. For instance, calling `do_math()` with integer or complex numbers will generate different code paths:

In [3]:
do_math.inspect_types()

In [4]:
%time do_math(1, 2)

CPU times: user 124 ms, sys: 26.5 ms, total: 151 ms
Wall time: 156 ms


3

In [5]:
%time do_math(1, 2)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.78 µs


3


**What is Numba doing to make code run quickly?**

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, after running (compiling) `do_math()` and you can access the `inspect_types` method.

In [6]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y




In [7]:
%time do_math(1j, 2)

CPU times: user 43.7 ms, sys: 66 µs, total: 43.8 ms
Wall time: 43 ms


(2+1j)

In [8]:
%time do_math(1j, 2)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 10.7 µs


(2+1j)

In [9]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y


do_math (complex128, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: complex128
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: complex128
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: complex128
    #   del $0.3
    #   return $0.4

    return x + y




### Eager compilation

- Tell Numba the function signature you are expecting

In [10]:
from numba import int32

In [11]:
@jit(int32(int32, int32))
def eager_do_math(x, y):
    return x + y

In [12]:
%time eager_do_math(1, 2)

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11 µs


3

In [13]:
%time eager_do_math(1.0, 2.0)

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 15.5 µs


3

In [14]:
%time eager_do_math(1j, 2)

TypeError: No matching definition for argument type(s) complex128, int64

## How does Numba work?

![](./images/how-does-numba-work.png)

Source: [Scaling Python Up and Out with Numba and Dask — Travis Oliphant](https://speakerdeck.com/teoliphant/scaling-python-up-and-out-with-numba-and-dask?slide=37)



### What about the actual LLVM code?
You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method. 

In [15]:
for key, value in do_math.inspect_llvm().items():
    print(key, value)

(int64, int64) ; ModuleID = 'do_math'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__11do_math$241Exx" = common local_unnamed_addr global i8* null
@.const.do_math = internal constant [8 x i8] c"do_math\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__11do_math$241Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.x, i64 %arg.y) local_unnamed_addr #0 {
entry:
  %.14 = add nsw i64 %arg.y, %arg.x
  store i64 %.14, i64* %retptr, align 8
  ret i32 0
}

define i8* @"_ZN7cpython8__main__11do_math$241Exx"(i8* nocapture readnone %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) local_unnamed_addr {
entry:
  %.5 = alloca i8*, align 8
  %.6 = alloca i8*, align 8
  %.7 = call i32 (i8*, i8*, i

**But there's a caveat....**

## Compilation Options

Numba has two compilation modes:

- **nopython mode (recommended and best-practice way)**: produces much faster code by running the code without the involvement of the Python interpreter.

- **object mode (should be avoided)**: Numba falls back to this mode when `nopython` mode fails.

To illustrate the above, let's watch what happens when we try to do something that is natural in Python (concatenating strings), but not particularly mathematically sound:

In [16]:
%time do_math('Hello', 'World')

CPU times: user 635 ms, sys: 589 µs, total: 636 ms
Wall time: 642 ms


'HelloWorld'

In [17]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y


do_math (complex128, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: complex128
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: complex128
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: complex128
    #   del $0.3
    #   return $0.4

    return x + y


do_math (unicode_type, unicode_type)
-

`do_math (unicode_type, unicode_type)` means that is has been compiled in `object` mode. 

To prevent Numba from falling back, and instead raise an error, we need to pass `nopython=True` to `@jit` decorator:

In [18]:
@jit
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    b = str(y)
    return a + b 

In [19]:
%timeit f(1, 2)

2.56 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
@jit(nopython=True) # Fore nopython mode
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    b = str(y)
    return a + b 

In [21]:
%timeit f(1, 2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'str': cannot determine Numba type of <class 'type'>

File "<ipython-input-20-d592c0643be0>", line 3:
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the error message
and traceback, along with a minimal reproducer at:
https://github.com/numba/numba/issues/new
